## Lesson 3 Exercise 3: Focus on the WHERE clause used to query a Composite Key

### Walk through the basics of creating a table in Apache Cassandra, inserting rows of data, and doing a simple CQL query to validate the information. You will practice coming up with queries which can be used using the WHERE clause, which field can be used as part of your WHERE clause and which of them you CANNOT, which is an encouraged practice with Apache Cassandra. 

In [100]:
# Import Cassandra driver to perform operations on Cassandra DB
import cassandra

In [101]:
# Import Pandas to import data from excel sheet
import pandas as pd

### Create a connection to the Cluster (equivalent to DB server in SQL)

In [102]:
from cassandra.cluster import Cluster
try:
    cluster = Cluster(['127.0.0.1'])
    session = cluster.connect()
except Exception as e:
    print(e)

### Create a keyspace we have to work with (equivalent to a Database in SQL)

In [103]:
try:
    session.execute("""
                    CREATE KEYSPACE IF NOT EXISTS udacity
                    WITH REPLICATION =
                    { 'class': 'SimpleStrategy', 'replication_factor': 1 } """)
except Exception as e:
    print(e)

### Connect to Keyspace. Comapre this with how we create a new session in Postgres

In [104]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### We would like to create a New OLAP table to store the Music Library of albums

<b> The Table contains the following columns, </b>
<ul>
    <li> 'Year' column as the <b>Primary key</b>. </li>
    <li> 'Month' and 'City' (Where Released) as the <b>Clustering columns</b>. </li>
    <li> 'Artist Name' and 'Album Name' as the other data columns. </li>
</ul>
</b>

![music_library](image5.png)

#### The following are the queries which the Business would like to use to analyze data.

#### 1. Give every album in the music library that was released in the given Year
`SELECT * FROM music_library WHERE year = 1970`

#### 2. Give every album in the music library that was released in the given Year and Month
`SELECT * FROM music_library WHERE year = 1970 AND month = 'July'`

#### 3. Give every album in the music library that was released in the given Year and Artist Name
`SELECT * FROM music_library WHERE year = 1970 AND artist_name = 'The Beatles'`

### DROP THE MUSIC LIBRARY table IF EXISTS

In [105]:
dropTable('music_library')

### CREATE TABLE IN KEYSPACE: udacity

In [34]:
try:
    query = """ CREATE TABLE IF NOT EXISTS music_library
                (year int, month varchar, city varchar, artist_name varchar, album_name text, 
                PRIMARY KEY(year, month, city)) """
    
    # CREATE TABLE MUSIC LIBRARY 
    session.execute(query)
    
except Exception as e:
    print(e)

### IMPORT THE EXCEL FILE with DATA (.xlsx)

In [35]:
df = pd.read_excel('Tables.xlsx', sheet_name='data', index_col=None)
df

,Year,Month,City,Artist_Name,Album_Name
0,1965,March,Oxford,The Beatles,Rubber Soul
1,1970,August,Liverpool,The Beatles,Let it Be
2,1966,June,Los Angeles,The Monkees,The Monkees
3,1970,July,San Diego,The Carpenters,Close To You
4,1964,January,London,The Beatles,Beatles For Sale
5,1970,July,Cleveland,Gonugunta's,Jai Srimanarayana


### DDL STATEMENT FOR CREATING THE TABLE

In [36]:
query = "INSERT INTO music_library (year, month, city, artist_name, album_name)"
query = query + " VALUES (%s, %s, %s, %s, %s)"

### INSERT DATA INTO THE TABLES IN KEYSPACE: udacity

In [54]:
# Extract Data from the DataFrame to Insert data to the Table
try:
    for index, row in df.iterrows():
        session.execute(query, list(row.values))
except Exception as e:
    print(e)

### INSERT DATA USING INDIVIDUAL INSERT STATEMENTS

try:
    session.execute(query, (1965, "March", "Oxford", "The Beatles", "Rubber Soul"))
except Exception as e:
    print(e)
    
try:
    session.execute(query, (1970, "August", "Liverpool", "The Beatles", "Let it Be"))
except Exception as e:
    print(e)
    
try:
    session.execute(query, (1964, "January", "London", "The Beatles", "Beatles for Sale"))
except Exception as e:
    print(e)

try:
    session.execute(query, (1966, "June", "Los Angeles", "The Monkees", "The Monkees"))
except Exception as e:
    print(e)

try:
    session.execute(query, (1970, "July", "San Diego", "The Carpenters", "Close To You"))
except Exception as e:
    print(e)
    
try:
    session.execute(query, (1970, "July", "Cleveland", "Gonugunta's", "Jai Srimanarayana"))
except Exception as e:
    print(e)

### Validate the Data Model of the Music Library table with WHERE clause = Primary Key

In [89]:
query = 'SELECT * FROM music_library WHERE year=1970'
try:
    rows = session.execute(query)
    
    # Display the result as DataFrame
    print(disp_df(rows))
    
except Exception as e:
    print(e)

   year   month       city         album_name     artist_name
0  1970  August  Liverpool          Let it Be     The Beatles
1  1970    July  Cleveland  Jai Srimanarayana     Gonugunta's
2  1970    July  San Diego       Close To You  The Carpenters


### Validate the Data Model of the Music Library table with WHERE clause = Primary + Clustering Key

In [92]:
try:
    query = "SELECT * FROM music_library WHERE year=1970 AND month='July'"
    rows = session.execute(query)
    
    # Display the result as DataFrame
    print(disp_df(rows))
    
except Exception as e:
    print(e)

   year month       city         album_name     artist_name
0  1970  July  Cleveland  Jai Srimanarayana     Gonugunta's
1  1970  July  San Diego       Close To You  The Carpenters


### Validate the Data Model of the Music Library table with WHERE clause = Primary + Clustering Key + Data Column

In [93]:
try:
    query = "SELECT * FROM music_library WHERE year=1970 AND month='July' AND album_name='Jai Srimanarayana'"
    rows = session.execute(query)
    
    # Display the result as DataFrame
    print(disp_df(rows))
    
except Exception as e:
    print(e)

Error from server: code=2200 [Invalid query] message="Cannot execute this query as it might involve data filtering and thus may have unpredictable performance. If you want to execute this query despite the performance unpredictability, use ALLOW FILTERING"


<div class="alert alert-block alert-warning">
    <b> Analysis </b>
    <ol>
        <li> The SELECT statement's WHERE clause contains has an non-clustering column which is <b>NOT</b> acceptable. </li>
        <li> The error is due to using the non-clustering column used and <b>NOT</b> all the clustering columns are used. </li>
    </ol>
    <b> We would have to take a re-look at our choice of data columns used in WHERE clause </b>
</div>

### Validate the Data Model of the Music Library table with WHERE clause = Primary + ALL Clustering Key

In [96]:
try:
    query = "SELECT * FROM music_library WHERE year=1970 AND month='July' AND city='Cleveland'"
    rows = session.execute(query)
    
    # Display the result as DataFrame
    print(disp_df(rows))
    
except Exception as e:
    print(e)

   year month       city         album_name  artist_name
0  1970  July  Cleveland  Jai Srimanarayana  Gonugunta's


<div class="alert alert-block alert-success">
    <b> Analysis of the Result Set </b>
    <ol>
    <li> The WHERE clause contains all the Clustering column keys. </li>
    <li> The result is in ascending order of the clustering column 'month' and 'city' (if multiple records were available). </li>
    </ol>
    <b> The Data Model and Analytic Queries look in a better shape now! </b>
</div>

### DROP THE MUSIC LIBRARY table IF EXISTS

In [33]:
dropTable('music_library')

### DISPLAY RESULTSET AS DATAFRAME

In [97]:
def disp_df(rows):
    
    # Extract the Columns of the ResultSet
    cols = rows.one()._fields
    # COnvert rows into list of lists
    data = [list(row) for row in rows]
    # Convert ResultSet into DataFrame
    df = pd.DataFrame(data,columns=cols)
    
    return df

### DROP TABLE FUNCTION

In [98]:
# Drop table from the KeySpace
def dropTable(table):
    try:
        query = "DROP TABLE IF EXISTS " + table
        session.execute(query)
        
    except Exception as e:
        print(e)

### Close the Session and Connection

In [99]:
session.shutdown()
cluster.shutdown()